# Problme 2

We are going to experiment with PyTorch’s DataParallel Module, which is PyTorch’s Synchronous SGD implementation across a number of GPUs on the same server. In particular, we will train ResNet-18 implementation from https://github.com/kuangliu/pytorch-cifar with num workers=2, running up to 4 GPUs with DataParallel (DP) Module. Use SGD optimizers with 0.1 as the learning rate, momentum 0.9, weight decay 5e-4. For this question, you need to do experiment with multiple GPUs on the same server. You may need to execute this on NYU Greene Cluster.

Create a PyTorch program with a DataLoader that loads the images and the related labels from torchvision CIFAR10 dataset. Import CIFAR10 dataset for the torchvision package, with the following sequence of transformations:

• Random cropping, with size 32x32 and padding 4

• Random horizontal flipping with a probability 0.5

• Normalize each image’s RGB channel with mean(0.4914, 0.4822, 0.4465) and variance (0.2023, 0.1994, 0.2010)

The DataLoader for the training set uses a minibatch size of 128 and 3 IO processes (i.e., num workers=2). The DataLoader for the testing set uses minibatch size of 100 and 3 IO processes (i.e., num workers =2). Create a main function that creates the DataLoaders for the training set and the neural network.

## 1
Measure how long does it take to compete 1 epoch of training using different batch size on a single GPU. Start from batch size 32, increase by 4-fold for each measurement (i.e., 32, 128, 512 ...) until single GPU memory cannot hold the batch size. For each run, run 2 epochs, the first epoch is used to warmup CPU/GPU cache; and you should report the training time (excluding data I/O; but including data movement from CPU to GPU, gradients calculation and weights update) based on the 2nd epoch training. (5)

In [319]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets

train_transform = transforms.Compose([transforms.RandomCrop(32, padding=4),#random crop
                                      transforms.RandomHorizontalFlip(0.5),#random flip
                                      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),#normalize
                                      transforms.ToTensor(),])#convert to tensor
train_dataset = datasets.CIFAR10("./cached_datasets/CIFAR10", train=True, download=True, transform=train_transform)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)

Files already downloaded and verified


**Please refer to the code in problem2 folder**

<center><img src="./problem2/2_1.png" width=500></center>

We use the straight line to calculate the time for each configuration.

In [2]:
import pandas as pd
data = pd.read_csv("./problem2/2_1.csv")

In [3]:
data

,Relative Time (Wall),SGD_dp_batch_size_8192_4gpus - _step,SGD_dp_batch_size_8192_4gpus - _step__MIN,SGD_dp_batch_size_8192_4gpus - _step__MAX,SGD_dp_batch_size_8192_4gpus - _timestamp,SGD_dp_batch_size_8192_4gpus - _timestamp__MIN,SGD_dp_batch_size_8192_4gpus - _timestamp__MAX,SGD_dp_batch_size_8192_4gpus - _runtime,SGD_dp_batch_size_8192_4gpus - _runtime__MIN,SGD_dp_batch_size_8192_4gpus - _runtime__MAX,...,SGD_dp_batch_size_8192_2gpus - _step__MAX,SGD_dp_batch_size_8192_2gpus - _timestamp,SGD_dp_batch_size_8192_2gpus - _timestamp__MIN,SGD_dp_batch_size_8192_2gpus - _timestamp__MAX,SGD_dp_batch_size_8192_2gpus - _runtime,SGD_dp_batch_size_8192_2gpus - _runtime__MIN,SGD_dp_batch_size_8192_2gpus - _runtime__MAX,SGD_dp_batch_size_8192_2gpus - epoch,SGD_dp_batch_size_8192_2gpus - epoch__MIN,SGD_dp_batch_size_8192_2gpus - epoch__MAX
0,NaN,7.5,0,15,1648372577,1648372568,1648372586,49,40,58,...,15,1.648371e+09,1648371067,1648371088,49.875,39,60,0.625,0,2


In [7]:
import wandb
import pandas as pd
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("xiang-pan/NYU_DL_Sys-HW3_problem2")
summary_list = [] 
config_list = [] 
name_list = [] 
res = {}
for run in runs: 
    data = run.history()
    log_name = run.name
    res[log_name] = data


In [8]:
res.keys()

dict_keys(['SGD_dp_batch_size_512_2gpus', 'SGD_dp_batch_size_2048_2gpus', 'SGD_dp_batch_size_8192_4gpus', 'SGD_dp_batch_size_2048_4gpus', 'SGD_dp_batch_size_512_4gpus', 'SGD_dp_batch_size_128_4gpus', 'SGD_dp_batch_size_32_4gpus', 'SGD_dp_batch_size_8192_1gpus', 'SGD_dp_batch_size_2048_1gpus', 'SGD_dp_batch_size_512_1gpus', 'SGD_dp_batch_size_128_1gpus', 'SGD_dp_batch_size_32_1gpus', 'SGD_dp_batch_size_8192_2gpus', 'SGD_dp_batch_size_128_2gpus', 'SGD_dp_batch_size_32_2gpus'])

In [39]:
res

{'SGD_dp_batch_size_512_2gpus':      trainer/global_step  _step  _runtime  train_loss_step  epoch  _timestamp  \
 0                      0      0        18         2.420080      0  1649128380   
 1                      1      1        19         2.769084      0  1649128381   
 2                      2      2        19         3.562219      0  1649128381   
 3                      3      3        19         3.776740      0  1649128381   
 4                      4      4        19         3.318298      0  1649128381   
 ..                   ...    ...       ...              ...    ...         ...   
 193                  192    193        37         1.172103      1  1649128399   
 194                  193    194        37         1.230016      1  1649128399   
 195                  194    195        37         1.342752      1  1649128399   
 196                  195    196        37         1.132757      1  1649128399   
 197                  195    197        38              NaN      2 

In [40]:
def get_info(log_name):
    gpu_nums = log_name.split('_')[-1]
    batch_size = log_name.split('_')[-2]
    batch_size = int(batch_size)
    gpu_nums = gpu_nums.replace('gpus','')
    gpu_nums = int(gpu_nums)
    return gpu_nums, batch_size

def get_time_info(log_name):
    data = res[log_name]
    epoch_1_start_time = data[data["epoch"] == 1].iloc[0]["_timestamp"]
    epoch_1_end_time = data[data["epoch"] == 1].iloc[-1]["_timestamp"]
    t = epoch_1_end_time - epoch_1_start_time
    return t
get_time_info("SGD_dp_batch_size_8192_4gpus")

12.0

batch_size = 8192 can not work in my gpu.

In [74]:
df = pd.DataFrame(columns=['gpu_nums', 'batch_size', 'time'])
for key in res.keys():
    # print(key)
    if "8192" in key:
        continue
    gpu_nums, batch_size = get_info(key)
    t = get_time_info(key)
    df.loc[len(df)] = [gpu_nums, batch_size, t]
df.sort_values(by=['gpu_nums','batch_size'], ascending=True, inplace=True)
df["gpu_nums"] = df["gpu_nums"].astype(int)
df["batch_size"] = df["batch_size"].astype(int)
df.reset_index(inplace=True, drop=True)

In [75]:
df.to_markdown("problem2/2_1_table_src.md")

In [76]:
df

,gpu_nums,batch_size,time
0,1,32,38.0
1,1,128,16.0
2,1,512,13.0
3,1,2048,13.0
4,2,32,67.0
5,2,128,18.0
6,2,512,9.0
7,2,2048,10.0
8,4,32,121.0
9,4,128,30.0


In [45]:
t = df[df["batch_size"] == 32]
t["speedup"] = t["time"].iloc[0]/t["time"] 
t

/tmp/ipykernel_768624/1156332872.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t["speedup"] = t["time"].iloc[0]/t["time"] * t["gpu_nums"]


,gpu_nums,batch_size,time,speedup
0,1,32,38.0,1.000000
4,2,32,67.0,1.134328
8,4,32,121.0,1.256198


In [77]:
t = df[df["batch_size"] == 128]
t["speedup"] = t["time"].iloc[0]/t["time"] 
t

/tmp/ipykernel_768624/3464596280.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t["speedup"] = t["time"].iloc[0]/t["time"]


,gpu_nums,batch_size,time,speedup
1,1,128,16.0,1.000000
5,2,128,18.0,0.888889
9,4,128,30.0,0.533333


In [78]:
t = df[df["batch_size"] == 512]
t["speedup"] = t["time"].iloc[0]/t["time"]
t

/tmp/ipykernel_768624/3178531423.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t["speedup"] = t["time"].iloc[0]/t["time"]


,gpu_nums,batch_size,time,speedup
2,1,512,13.0,1.000000
6,2,512,9.0,1.444444
10,4,512,11.0,1.181818


In [79]:
t = df[df["batch_size"] == 2048]
t["speedup"] = t["time"].iloc[0]/t["time"]
t

/tmp/ipykernel_768624/2197254454.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t["speedup"] = t["time"].iloc[0]/t["time"]


,gpu_nums,batch_size,time,speedup
3,1,2048,13.0,1.000000
7,2,2048,10.0,1.300000
11,4,2048,11.0,1.181818


In [80]:
l = []
for b in [32, 128, 512, 2048]:
    t = df[df["batch_size"] == b]
    t["speedup"] = t["time"].iloc[0]/t["time"]
    t.to_markdown(f"problem2/2_1_table_src_{b}.md")
    l.append(t)
l = pd.concat(l)

/tmp/ipykernel_768624/2183533066.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t["speedup"] = t["time"].iloc[0]/t["time"]


In [81]:
l

,gpu_nums,batch_size,time,speedup
0,1,32,38.0,1.000000
4,2,32,67.0,0.567164
8,4,32,121.0,0.314050
1,1,128,16.0,1.000000
5,2,128,18.0,0.888889
9,4,128,30.0,0.533333
2,1,512,13.0,1.000000
6,2,512,9.0,1.444444
10,4,512,11.0,1.181818
3,1,2048,13.0,1.000000


In [82]:
l[l["batch_size"] == b]

,gpu_nums,batch_size,time,speedup
3,1,2048,13.0,1.000000
7,2,2048,10.0,1.300000
11,4,2048,11.0,1.181818


In [83]:
# pd.set_option('max_columns',1000)
cols = []
cols.append("gpu_nums")
for b in [32, 128, 512, 2048]:
    cols.append(f"batch_{b}_time")
    cols.append(f"batch_{b}_speedup")
# print(cols)
df = pd.DataFrame(columns=cols)
for g in [1, 2, 4]:
    temp_df = pd.DataFrame(columns=cols)
    temp_df["gpu_nums"] = [g]
    for b in [32, 128, 512, 2048]:
        t = l[l["batch_size"] == b]
        t = t[t["gpu_nums"] == g]
        temp_df[f"batch_{b}_time"] = t["time"].values
        temp_df[f"batch_{b}_speedup"] = t["speedup"].values
    df = pd.concat([df, temp_df])
df.reset_index(inplace=True, drop=True)
df

,gpu_nums,batch_32_time,batch_32_speedup,batch_128_time,batch_128_speedup,batch_512_time,batch_512_speedup,batch_2048_time,batch_2048_speedup
0,1,38.0,1.000000,16.0,1.000000,13.0,1.000000,13.0,1.000000
1,2,67.0,0.567164,18.0,0.888889,9.0,1.444444,10.0,1.300000
2,4,121.0,0.314050,30.0,0.533333,11.0,1.181818,11.0,1.181818


In [84]:
df.to_markdown("problem2/2_1_table_src_all.md")
df.to_csv("problem2/2_1_table_src_all.csv", index=False)

That speed up we use T = epoch_time, **T(1) / T(N)**. 

According to the [campuswire](https://campuswire.com/c/G84B0471C/feed/127), we are suggested to use N * T(1) / T(N). But the not sure the T definition here. 

To clarify, we use t = batch_time and T = epoch_time.

We can calculate using n * t(1) / t(N) if we would like to use the same problem size.

Speedup = T(1) / T(N) = n * t(1) / t(N),   T is the epoch time, t is the batch time.

|    |   gpu_nums |   batch_32_time |   batch_32_speedup |   batch_128_time |   batch_128_speedup |   batch_512_time |   batch_512_speedup |   batch_2048_time |   batch_2048_speedup |
|---:|-----------:|----------------:|-------------------:|-----------------:|--------------------:|-----------------:|--------------------:|------------------:|---------------------:|
|  0 |          1 |              38 |           1        |               16 |            1        |               13 |             1       |                13 |              1       |
|  1 |          2 |              67 |           0.567164 |               18 |            0.888889 |                9 |             1.44444 |                10 |              1.3     |
|  2 |          4 |             121 |           0.31405  |               30 |            0.533333 |               11 |             1.18182 |                11 |              1.18182 |

If we use the **N * T(1) / T(N)** definition, we can get the following result:

In [86]:
new_df = df.copy(deep=True)

for b in [32, 128, 512, 2048]:
    new_df[f"batch_{b}_speedup"] = new_df[f"batch_{b}_speedup"] * new_df["gpu_nums"]
new_df

,gpu_nums,batch_32_time,batch_32_speedup,batch_128_time,batch_128_speedup,batch_512_time,batch_512_speedup,batch_2048_time,batch_2048_speedup
0,1,38.0,1.0,16.0,1.0,13.0,1.0,13.0,1.0
1,2,67.0,1.134328,18.0,1.777778,9.0,2.888889,10.0,2.6
2,4,121.0,1.256198,30.0,2.133333,11.0,4.727273,11.0,4.727273


## 2
Report for each batch size per gpu (i.e., 32, 128, 512 ...), how much time spent in computation (including CPU-GPU transferring and calculation) and how much time spent in communication in 2-GPU and 4-GPU case for one epoch. (hint You could use the training time reported in Question 1 to facilitate your calculation). (5) Expected Answer: First, describe how do you get the compute and communication time in each setup. Second, list compute and communication time in Table 2.

### Comment
Expected Answer: Table 1 records the training time and speedup for different batch size up to 4 GPUs. Comment on which type of scaling we are measuring: weak-scaling or strong-scaling? Comment on if the other type scaling was used speedup number will be better or worse than what you we are measuring.

If we follow the definition, T is the epoch time, and we use T(1) / T(N) to calculate the speedup, the we use the stong-scaling (Strong scaling concerns the speedup for a fixed problem size with respect to the number of processors, and is governed by Amdahl’s law) to measure the speedup, because we measure the epoch time, and the problem size for one epoch is fixed.


\begin{equation}
    \text { Speedup }= 1 /(s+p / N),
\end{equation}

s is the serial part, p is the parallel part, N is the number of processors.

\begin{equation}
    \text{Speedup}= T(1) / T(N)
\end{equation}


If we use the weak-scaling (Weak scaling concerns the speedup for a scaled problem size with respect to the number of processors, and is governed by Gustafson’s law.) to measure the speedup.

\begin{equation}
\text { Speedup }=s+p * N
\end{equation}

\begin{equation}
\text { Speedup }=N * T(1) / T(N)
\end{equation}

Considering the larger problem size will be more efficient for more processors, if we use the weak-scaling to measure the speedup, we will get a better speedup efficiency.

### Suggested Definition

If we use the **N * T(1) / T(N)** definition, T is the epoch time, then we are using the weak-scaling to measure the speedup, because we vary the problem size with respect to the number of processors.

Then if we use another definition (strong scaling), the score will be worse.

## 3
Report for each batch size per gpu (i.e., 32, 128, 512 ...), how much time spent in computation (including CPU-GPU transferring and calculation) and how much time spent in communication in 2-GPU and 4-GPU case for one epoch. (hint You could use the training time reported in Question 1 to facilitate your calculation). (5) Expected Answer: First, describe how do you get the compute and communication time in each setup. Second, list compute and communication time in Table 2.

For one gpu, we consider all the time is used for computation (including CPU-GPU transferring and calculation).

For two gpus or four gpus, the iteration number for each gpu is reduced, thus the computation time is reduced half of the original time, the communication time is the difference between the actual time and the reduced computation time.

In [61]:
df = pd.read_csv("problem2/2_1_table_src_all.csv")
cal_col = []
for b in [32, 128, 512, 2048]:
    df.drop(columns=[f"batch_{b}_speedup"], inplace=True)
    cal_col.append(f"batch_{b}_time")

In [62]:
df

,gpu_nums,batch_32_time,batch_128_time,batch_512_time,batch_2048_time
0,1,38.0,16.0,13.0,13.0
1,2,67.0,18.0,9.0,10.0
2,4,121.0,30.0,11.0,11.0


In [89]:
communication_col = []
calculation_col = []
for b in [32, 128, 512, 2048]:
    communication_col.append(f"batch_{b}_communication_time")
    calculation_col.append(f"batch_{b}_calculation_time")

In [90]:
# df["calculation_time"
communication_time = []
calculation_time = []
for g in [1, 2, 4]:
    if g == 1:
        temp_df = df[df["gpu_nums"] == g]
        temp_df = temp_df[cal_col].to_numpy()[0]
        base_df = temp_df
        communication_time.append(temp_df-temp_df)
        calculation_time.append(base_df)
        continue
    temp_df = df[df["gpu_nums"] == g]
    temp_df = temp_df[cal_col].to_numpy()[0]
    calculation_time.append(base_df / g)
    res_df = temp_df - base_df / g
    communication_time.append(res_df)
df[communication_col] = communication_time
df[calculation_col] = calculation_time

In [92]:
df

,gpu_nums,batch_32_time,batch_32_speedup,batch_128_time,batch_128_speedup,batch_512_time,batch_512_speedup,batch_2048_time,batch_2048_speedup,batch_32_communication_time,batch_128_communication_time,batch_512_communication_time,batch_2048_communication_time,batch_32_calculation_time,batch_128_calculation_time,batch_512_calculation_time,batch_2048_calculation_time
0,1,38.0,1.000000,16.0,1.000000,13.0,1.000000,13.0,1.000000,0.0,0.0,0.00,0.00,38.0,16.0,13.00,13.00
1,2,67.0,0.567164,18.0,0.888889,9.0,1.444444,10.0,1.300000,48.0,10.0,2.50,3.50,19.0,8.0,6.50,6.50
2,4,121.0,0.314050,30.0,0.533333,11.0,1.181818,11.0,1.181818,111.5,26.0,7.75,7.75,9.5,4.0,3.25,3.25


In [93]:
df.to_csv('./problem2/2_3.csv', index=False)
df.to_markdown('./problem2/2_3.md')

In [96]:
display_cols = ["gpu_nums"] + communication_col
df[display_cols]

,gpu_nums,batch_32_communication_time,batch_128_communication_time,batch_512_communication_time,batch_2048_communication_time
0,1,0.0,0.0,0.00,0.00
1,2,48.0,10.0,2.50,3.50
2,4,111.5,26.0,7.75,7.75


In [97]:
display_cols = ["gpu_nums"] + calculation_col
df[display_cols]

,gpu_nums,batch_32_calculation_time,batch_128_calculation_time,batch_512_calculation_time,batch_2048_calculation_time
0,1,38.0,16.0,13.00,13.00
1,2,19.0,8.0,6.50,6.50
2,4,9.5,4.0,3.25,3.25


## 4
Assume PyTorch DP implements the all-reduce algorithm as discussed in the class (reference below), calculate communication bandwidth utilization for each multi-gpu/batch-size-per-gpu setup. (5) Expected Answer: First, list the formula to calculate how long does it take to finish an allreduce. Second, list the formula to calculate the bandwidth utilization. Third, list the calculated results in Table 3.

References:

• PyTorch Data Parallel, Available at https://pytorch.org/docs/stable/modules/torch/nn/parallel/data_parallel.html.

• Bringing HPC Techniques to Deep Learning

### Comment

We calculate the all-reduce (ring-allreduce).

P: number of processes 

N: total number of model parameters

Scatter-reduce: Each process sends N/P amount of data to (P-1) learners, Total amount sent (per process): N(P-1)/P

AllGather: Each process again sends N/P amount of data to (P-1) learners

Total communication cost per process is 2N(P-1)/P

|  | Batch-size-per-GPU 32 | Batch-size-per-GPU 128 | Batch-size-per-GPU 512 |
| :--- | :--- | :--- | :--- |
|  | Bandwidth Utilization(GB/s) | Bandwidth Utilization(GB/s) | Bandwidth Utilization(GB/s) |
| 2-GPU |  |  |  |
| 4-GPU |  |  |  |

We use the ResNet18 architecture to calculate the bandwidth utilization.

In [66]:
from models.resnet import ResNet18
import numpy as np


model = ResNet18()

# calculate model parameters size
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params_count = sum([np.prod(p.size()) for p in model_parameters])
print(f"Model parameters: {params_count}, {params_count/1e6}M")

N = params_count

Model parameters: 11173962, 11.173962M


In [67]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets

train_transform = transforms.Compose([transforms.RandomCrop(32, padding=4),#random crop
                                      transforms.RandomHorizontalFlip(0.5),#random flip
                                      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),#normalize
                                      transforms.ToTensor(),])#convert to tensor
train_dataset = datasets.CIFAR10("./cached_datasets/CIFAR10", train=True, download=True, transform=train_transform)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
dataset_size = len(train_dataset)

Files already downloaded and verified


In [68]:
df

,gpu_nums,batch_32_time,batch_128_time,batch_512_time,batch_2048_time,batch_32_communication_time,batch_128_communication_time,batch_512_communication_time,batch_2048_communication_time
0,1,38.0,16.0,13.0,13.0,0.0,0.0,0.00,0.00
1,2,67.0,18.0,9.0,10.0,48.0,10.0,2.50,3.50
2,4,121.0,30.0,11.0,11.0,111.5,26.0,7.75,7.75


In [69]:
bandwidth_cols = []
communication_count_cols = []
for b in [32, 128, 512, 2048]:
    bandwidth_cols.append(f"batch_{b}_bandwidth")
    communication_count_cols.append(f"batch_{b}_communication_count")
df[bandwidth_cols] = np.nan
df[communication_count_cols] = np.nan

In [70]:
import math
append_df = []
# P = 1

for i,P in enumerate([1, 2, 4]):
    if i == 0:
        continue
    for b in [32, 128, 512, 2048]:
        iter_num = math.ceil(dataset_size/b)
        each_iter_communication_count = 2 * N * (P - 1) / P
        communication_count = iter_num * each_iter_communication_count
        communication_time = df["batch_" + str(b) + "_communication_time"][i]
        # print(communication_time)
        # print(f"communication_count: {communication_count}")
        bandwidth_utilization = communication_count / (communication_time * 1e9)
        df["batch_" + str(b) + "_bandwidth"][i] = bandwidth_utilization
        df["batch_" + str(b) + "_communication_count"][i] = communication_count

/tmp/ipykernel_768624/3602963460.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["batch_" + str(b) + "_bandwidth"][i] = bandwidth_utilization
/tmp/ipykernel_768624/3602963460.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["batch_" + str(b) + "_communication_count"][i] = communication_count


In [71]:
df

,gpu_nums,batch_32_time,batch_128_time,batch_512_time,batch_2048_time,batch_32_communication_time,batch_128_communication_time,batch_512_communication_time,batch_2048_communication_time,batch_32_bandwidth,batch_128_bandwidth,batch_512_bandwidth,batch_2048_bandwidth,batch_32_communication_count,batch_128_communication_count,batch_512_communication_count,batch_2048_communication_count
0,1,38.0,16.0,13.0,13.0,0.0,0.0,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,67.0,18.0,9.0,10.0,48.0,10.0,2.50,3.50,0.363852,0.436902,0.438019,0.079814,1.746490e+10,4.369019e+09,1.095048e+09,279349050.0
2,4,121.0,30.0,11.0,11.0,111.5,26.0,7.75,7.75,0.234954,0.252059,0.211945,0.054068,2.619735e+10,6.553529e+09,1.642572e+09,419023575.0


In [72]:
df[communication_count_cols]

,batch_32_communication_count,batch_128_communication_count,batch_512_communication_count,batch_2048_communication_count
0,NaN,NaN,NaN,NaN
1,1.746490e+10,4.369019e+09,1.095048e+09,279349050.0
2,2.619735e+10,6.553529e+09,1.642572e+09,419023575.0


No communication for gpu_nums = 1

In [73]:
display_cols = ["gpu_nums"] + bandwidth_cols
df[display_cols]

,gpu_nums,batch_32_bandwidth,batch_128_bandwidth,batch_512_bandwidth,batch_2048_bandwidth
0,1,NaN,NaN,NaN,NaN
1,2,0.363852,0.436902,0.438019,0.079814
2,4,0.234954,0.252059,0.211945,0.054068


The bandwidth utilization is calculated by the following formula:
1. Calculate the iteration numer
2. Calculate the communication cost per process per iter.
3. Calculate the communication cost per process per epoch.
4. Using the epoch communication time to get the bandwidth utilization.

The above table unit is GB/s.

\begin{align}
T_{train} = T_{communication}+T_{compute} \\

T_{compute} = T_1/N \\

T_{communication} = T_{train} - T_1/N 
\end{align}

We have the T_{communication}, and we have all-reduce to get the model transfer size, we can get the bandwidth utilization by 

\begin{equation}
    \text{Bandwidth Utilization}= \frac{Size_{communication}}{T_{communication}}
\end{equation}